In [1]:
import pandas as pd
import matplotlib as plt
import datetime
import time
import numpy as np
import gc

In [2]:
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)

# Preprocess News Data

In [3]:
#Using 2010 to 2016 data
def process_news_data(news_data):
    print("Shape : {}".format(news_data.shape))
    news_data = news_data.drop(news_data.columns[0], axis=1)
    news_data['time'] = pd.to_datetime(news_data['time'])
    news_data['sourceTimestamp'] = pd.to_datetime(news_data['sourceTimestamp'])
    news_data['firstCreated'] = pd.to_datetime(news_data['firstCreated'])
    news_data['provider'] = news_data['provider'].astype('category')
    news_data['subjects'] = news_data['subjects'].astype('category')
    news_data['audiences'] = news_data['audiences'].astype('category')
    news_data['assetCodes'] = news_data['assetCodes'].astype('category')
    news_data['assetName'] = news_data['assetName'].astype('category')
    news_data = news_data[news_data['time'].dt.year > 2009]
    
    news_data['rel_firstMention'] = 1.0*news_data['firstMentionSentence']/news_data['sentenceCount']
    news_data['rel_SentCount'] = 1.0*news_data['sentimentWordCount']/news_data['wordCount']
    
    return news_data

In [4]:
df1 = pd.read_csv("train_news/news_train1.csv")
df1 = process_news_data(df1)
df2 = pd.read_csv("train_news/news_train2.csv")
df2 = process_news_data(df2)
df3 = pd.read_csv("train_news/news_train3.csv")
df3 = process_news_data(df3)
df4 = pd.read_csv("train_news/news_train4.csv")
df4 = process_news_data(df4)
df5 = pd.read_csv("train_news/news_train5.csv")
df5 = process_news_data(df5)
df6 = pd.read_csv("train_news/news_train6.csv")
df6 = process_news_data(df6)
df7 = pd.read_csv("train_news/news_train7.csv")
df7 = process_news_data(df7)
df8 = pd.read_csv("train_news/news_train8.csv")
df8 = process_news_data(df8)
df9 = pd.read_csv("train_news/news_train9.csv")
df9 = process_news_data(df9)
df10 = pd.read_csv("train_news/news_train10.csv")
df10 = process_news_data(df10)
news_data = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10])
# news_data = pd.concat([df1,df2,df3,df4])

Shape : (1000000, 36)
Shape : (1000000, 36)
Shape : (1000000, 36)
Shape : (1000000, 36)
Shape : (1000000, 36)
Shape : (1000000, 36)
Shape : (1000000, 36)
Shape : (1000000, 36)
Shape : (1000000, 36)
Shape : (328749, 36)


In [5]:
news_data = news_data[pd.notnull(news_data['headline'])]
news_data.shape

(6987566, 37)

In [6]:
## Removing late report of news
news_data['news_delay'] = news_data['time'] - news_data['sourceTimestamp']
news_data = news_data[news_data.news_delay < datetime.timedelta(days=1)]
news_data.shape

(6987562, 38)

In [7]:
news_data['time'] = news_data['time'].dt.date

# Analysis

In [8]:
news_data.head()

,time,sourceTimestamp,firstCreated,sourceId,headline,urgency,takeSequence,provider,subjects,audiences,bodySize,companyCount,headlineTag,marketCommentary,sentenceCount,wordCount,assetCodes,assetName,firstMentionSentence,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentWordCount,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D,rel_firstMention,rel_SentCount,news_delay
325350,2010-01-01,2010-01-01 00:39:51,2010-01-01 00:38:58,9254ca57fc256c11,FISHER COMMUNICATIONS SAYS DETAILS REGARDING T...,1,3,RTRS,"{'RET', 'PUB', 'BUS', 'US', 'RTRS', 'LEN'}","{'E', 'U'}",0,1,NaN,False,1,13,{'FSCI.O'},Sinclair Television of Seattle Inc,1,1.0,1,0.081374,0.315028,0.603598,13,0,0,0,0,0,1,1,1,1,1,1.000000,1.000000,0 days
325351,2010-01-01,2010-01-01 00:40:11,2010-01-01 00:38:58,37f1ed000e33ebe9,"BRIEF-Bright house networks, Fisher Communicat...",3,1,RTRS,"{'RET', 'PUB', 'BUS', 'US', 'RTRS', 'LEN'}","{'PCO', 'PCU', 'U', 'RNP', 'E'}",482,1,BRIEF,False,4,102,{'FSCI.O'},Sinclair Television of Seattle Inc,1,1.0,1,0.030909,0.118454,0.850637,100,2,2,2,2,2,2,2,2,2,2,0.250000,0.980392,0 days
325352,2010-01-01,2010-01-01 00:42:02,2010-01-01 00:42:02,c68c02d980a366b8,Willis Completes Change in Place of Incorporat...,3,1,BSW,"{'EUROPE', 'IE', 'INS', 'NEWR', 'INSR', 'BM', ...","{'BSW', 'CNR', 'CNRE'}",5294,1,NaN,False,28,867,{'WSH.N'},Willis Towers Watson PLC,1,1.0,0,0.171227,0.425594,0.403179,633,0,0,0,0,0,0,1,1,1,1,0.035714,0.730104,0 days
325353,2010-01-01,2010-01-01 00:44:13,2010-01-01 00:43:13,190b5184fbbf3e0d,WILLIS GROUP HOLDINGS LTD <WSH.N> SAYS REORGAN...,1,2,RTRS,"{'RTRS', 'US', 'INS', 'LEN'}","{'E', 'U'}",0,1,NaN,False,2,15,{'WSH.N'},Willis Towers Watson PLC,1,1.0,1,0.007043,0.438746,0.554210,15,1,1,1,1,1,1,2,2,2,2,0.500000,1.000000,0 days
325354,2010-01-01,2010-01-01 00:44:34,2010-01-01 00:43:13,bf0f8bd10bccd2d7,BRIEF-Willis completes change in place of inco...,3,1,RTRS,"{'RTRS', 'US', 'INS', 'LEN'}","{'PCO', 'PCU', 'U', 'RNP', 'E'}",313,1,BRIEF,False,4,79,{'WSH.N'},Willis Towers Watson PLC,1,1.0,0,0.012544,0.827926,0.159530,79,1,1,1,1,1,2,3,3,3,3,0.250000,1.000000,0 days


In [9]:
news_data.columns

Index(['time', 'sourceTimestamp', 'firstCreated', 'sourceId', 'headline',
       'urgency', 'takeSequence', 'provider', 'subjects', 'audiences',
       'bodySize', 'companyCount', 'headlineTag', 'marketCommentary',
       'sentenceCount', 'wordCount', 'assetCodes', 'assetName',
       'firstMentionSentence', 'relevance', 'sentimentClass',
       'sentimentNegative', 'sentimentNeutral', 'sentimentPositive',
       'sentimentWordCount', 'noveltyCount12H', 'noveltyCount24H',
       'noveltyCount3D', 'noveltyCount5D', 'noveltyCount7D', 'volumeCounts12H',
       'volumeCounts24H', 'volumeCounts3D', 'volumeCounts5D', 'volumeCounts7D',
       'rel_firstMention', 'rel_SentCount', 'news_delay'],
      dtype='object')

In [10]:
news_columns = ['time','urgency', 'assetCodes', 'assetName', 'relevance', 'sentimentClass','rel_firstMention', 
                'firstMentionSentence', 'sentenceCount','rel_SentCount', 'sentimentWordCount', 'wordCount']

In [11]:
news_data = news_data[news_columns]

In [12]:
gc.collect()

0

In [ ]:
#news_data['time'] = news_data.time.dt.date

In [ ]:
#news_data.groupby(['time', 'assetName'])['sentimentNegative','sentimentNeutral', 'sentimentPositive'].sum()

In [13]:
tmp = news_data.groupby(['time', 'assetName'])['sentimentClass'].nunique()

In [14]:
print("Combination of days and assets : {}".format(tmp.count()))
print("Total sentiment counts for these combinations : {}".format(tmp.sum()))

Combination of days and assets : 1602799
Total sentiment counts for these combinations : 2480771


There are total 2489774 sentiment counts, spanning 1608251 combination of days and assets.
Taking average will change the actual sentiment of the aggregated news.
Need to find a way of weighted aggregating the news data.

# Preparing news data

In [15]:
news_data.columns

Index(['time', 'urgency', 'assetCodes', 'assetName', 'relevance',
       'sentimentClass', 'rel_firstMention', 'firstMentionSentence',
       'sentenceCount', 'rel_SentCount', 'sentimentWordCount', 'wordCount'],
      dtype='object')

In [16]:
news_data.groupby('urgency').describe()

firstMentionSentence                                                  \
                       count      mean        std  min  25%  50%  75%    max   
urgency                                                                        
1                  2547603.0  0.775600   0.417187  0.0  1.0  1.0  1.0    1.0   
2                       25.0  1.600000   2.449490  0.0  1.0  1.0  1.0   12.0   
3                  4439934.0  7.509035  16.082603  0.0  1.0  1.0  7.0  989.0   

        rel_SentCount                                                    \
                count      mean       std       min       25%       50%   
urgency                                                                   
1           2547603.0  0.999350  0.017242  0.071429  1.000000  1.000000   
2                25.0  0.842614  0.198373  0.214612  0.781250  0.869565   
3           4439934.0  0.540777  0.372102  0.000667  0.148387  0.562269   

                       rel_firstMention                                     \
              75%  max            count      mean       std  min       25%   
urgency                                                                      
1        1.000000  1.0        2547603.0  0.762665  0.418052  0.0  1.000000   
2        1.000000  1.0             25.0  0.247311  0.213494  0.0  0.142857   
3        0.935728  1.0        4439934.0  0.217514  0.261948  0.0  0.027778   

                                       relevance                      \
              50%       75%       max      count      mean       std   
urgency                                                                
1        1.000000  1.000000  1.000000  2547603.0  0.980980  0.076657   
2        0.200000  0.400000  0.666667       25.0  0.731033  0.348767   
3        0.090909  0.333333  1.000000  4439934.0  0.633924  0.411307   

                                           sentenceCount             \
              min       25%  50%  75%  max         count       mean   
urgency                                                               
1        0.200000  1.000000  1.0  1.0  1.0     2547603.0   1.084559   
2        0.123091  0.316228  1.0  1.0  1.0          25.0   5.800000   
3        0.002409  0.174078  1.0  1.0  1.0     4439934.0  35.562190   

                                                  sentimentClass            \
               std  min   25%   50%   75%     max          count      mean   
urgency                                                                      
1         0.288991  1.0   1.0   1.0   1.0     8.0      2547603.0  0.043532   
2         3.109126  2.0   4.0   5.0   6.0    19.0           25.0  0.480000   
3        42.933997  1.0  10.0  24.0  43.0  1205.0      4439934.0  0.130331   

                                           sentimentWordCount              \
              std  min  25%  50%  75%  max              count        mean   
urgency                                                                     
1        0.740681 -1.0 -1.0  0.0  1.0  1.0          2547603.0   16.992980   
2        0.822598 -1.0  0.0  1.0  1.0  1.0               25.0  136.720000   
3        0.873516 -1.0 -1.0  0.0  1.0  1.0          4439934.0  322.322732   

                                                        wordCount              \
                std   min   25%    50%    75%      max      count        mean   
urgency                                                                         
1          5.785137   1.0  13.0   16.0   21.0     71.0  2547603.0   17.008057   
2         74.778852  47.0  91.0  105.0  163.0    302.0       25.0  167.120000   
3        449.252874   2.0  86.0  163.0  389.0  13087.0  4439934.0  937.925816   

                                                           
                 std   min    25%    50%     75%      max  
urgency                                                    
1           5.793574   1.0   13.0   16.0    21.0     71.0  
2          91.954391  75.0  115.0  136.0   180.0    477.0  
3        1146.343967   2.0  

In [19]:
news_data = news_data[(news_data['urgency'] != 2)]

In [20]:
news_data.groupby('urgency').describe()

firstMentionSentence                                                  \
                       count      mean        std  min  25%  50%  75%    max   
urgency                                                                        
1                  2547603.0  0.775600   0.417187  0.0  1.0  1.0  1.0    1.0   
3                  4439934.0  7.509035  16.082603  0.0  1.0  1.0  7.0  989.0   

        rel_SentCount                                                    \
                count      mean       std       min       25%       50%   
urgency                                                                   
1           2547603.0  0.999350  0.017242  0.071429  1.000000  1.000000   
3           4439934.0  0.540777  0.372102  0.000667  0.148387  0.562269   

                       rel_firstMention                                     \
              75%  max            count      mean       std  min       25%   
urgency                                                                      
1        1.000000  1.0        2547603.0  0.762665  0.418052  0.0  1.000000   
3        0.935728  1.0        4439934.0  0.217514  0.261948  0.0  0.027778   

                                  relevance                                \
              50%       75%  max      count      mean       std       min   
urgency                                                                     
1        1.000000  1.000000  1.0  2547603.0  0.980980  0.076657  0.200000   
3        0.090909  0.333333  1.0  4439934.0  0.633924  0.411307  0.002409   

                                 sentenceCount                             \
              25%  50%  75%  max         count       mean        std  min   
urgency                                                                     
1        1.000000  1.0  1.0  1.0     2547603.0   1.084559   0.288991  1.0   
3        0.174078  1.0  1.0  1.0     4439934.0  35.562190  42.933997  1.0   

                                  sentimentClass                           \
          25%   50%   75%     max          count      mean       std  min   
urgency                                                                     
1         1.0   1.0   1.0     8.0      2547603.0  0.043532  0.740681 -1.0   
3        10.0  24.0  43.0  1205.0      4439934.0  0.130331  0.873516 -1.0   

                            sentimentWordCount                               \
         25%  50%  75%  max              count        mean         std  min   
urgency                                                                       
1       -1.0  0.0  1.0  1.0          2547603.0   16.992980    5.785137  1.0   
3       -1.0  0.0  1.0  1.0          4439934.0  322.322732  449.252874  2.0   

                                      wordCount                                \
          25%    50%    75%      max      count        mean          std  min   
urgency                                                                         
1        13.0   16.0   21.0     71.0  2547603.0   17.008057     5.793574  1.0   
3        86.0  163.0  389.0  13087.0  4439934.0  937.925816  1146.343967  2.0   

                                        
           25%    50%     75%      max  
urgency                                 
1         13.0   16.0    21.0     71.0  
3        263.0  615.0  1116.0  20263.0

In [22]:
range_sent = news_data['sentimentClass'].unique()
range_urg = news_data['urgency'].unique()
print(range_urg)
print(range_sent)

[1 3]
[ 1  0 -1]


In [23]:
columns = [ 'relevance', 'rel_firstMention', 'firstMentionSentence',
       'sentenceCount', 'rel_SentCount', 'sentimentWordCount', 'wordCount']

In [25]:
for i in range_sent:
    for j in range_urg:
        for col in columns:
            new_col = col + "_" + str(j) + '_' + str(i)
            news_data[new_col] = 0.0
            news_data.loc[((news_data['sentimentClass'] == i)  & (news_data['urgency'] == j)),new_col] = news_data.loc[((news_data['sentimentClass'] == i)  & (news_data['urgency'] == j)),col]
news_data.drop(labels=columns + ['relevance','sentimentClass'], axis=1, inplace=True)

MemoryError: 

In [ ]:
news_data.shape

In [ ]:
news_data.columns

In [ ]:
gc.collect()

In [ ]:
news_data.columns

### Preprocess Market Data 

In [ ]:
df1 = pd.read_csv("train_market/market_train1.csv")
df2 = pd.read_csv("train_market/market_train2.csv")
df3 = pd.read_csv("train_market/market_train3.csv")
df4 = pd.read_csv("train_market/market_train4.csv")
df5 = pd.read_csv("train_market/market_train5.csv")
market_data = pd.concat([df1,df2,df3,df4,df5])

In [ ]:
print(sorted(news_data['firstMentionSentence'].unique()))

In [ ]:
news_data[news_data['assetName'] == 'Google Inc'].head(20)

In [ ]:
news_data.head()

In [ ]:
news_data.groupby('urgency').describe()

# Rough Work

In [ ]:
df1 = pd.DataFrame({'a':['dfg','f','fff','fgrf','fghj'], 'b' : ['sd','dfg','edr','df','fghjky']})

df1['c'] = df1.apply(lambda x: max([len(x) for x in [df1['a'], df1['b']]]))


In [ ]:
df1

In [ ]:
news_data[(news_data['assetName'] == "Cablevision Systems Corp") & (news_data['time'] == datetime.datetime.strptime('2010-01-01', "%Y-%m-%d").date())]

In [ ]:
news_group = news_data.groupby(['time', 'assetName'])

In [ ]:
news_group[]

In [ ]:
def unstack_asset_codes(news_train):
    codes = []
    indexes = []
    for i, values in news_train['assetCodes'].iteritems():
        explode = values.split(", ")
        codes.extend(explode)
        repeat_index = [int(i)]*len(explode)
        indexes.extend(repeat_index)
    index_df = pd.DataFrame({'news_index': indexes, 'assetCode': codes})
    del codes, indexes
    gc.collect()
    return index_df

In [ ]:
index_df = unstack_asset_codes(news_data)
index_df.head()

In [ ]:
def merge_news_on_index(news_train, index_df):
    news_train['news_index'] = news_train.index.copy()

    # Merge news on unstacked assets
    news_unstack = index_df.merge(news_train, how='left', on='news_index')
    news_unstack.drop(['news_index', 'assetCodes'], axis=1, inplace=True)
    return news_unstack

In [ ]:
news_unstack = merge_news_on_index(news_data, index_df)
del news_train, index_df
gc.collect()
news_unstack.head(3)